In [1]:
!pip install -q pogema==1.2.2
!pip install -q stable_baselines3==2.1.0
!pip install -q sb3-contrib==2.1.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 80.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow 2.14.0 requires numpy>=1.23.5, but you have numpy 1.23.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
import gymnasium as gym
from pogema import GridConfig

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy

In [4]:
grid_config = GridConfig(
    size=8,
    density=0.3,
    num_agents=1,
    observation_radius = 5,
    max_episode_steps=64
)

env = gym.make("Pogema-v0",grid_config=grid_config)

# 新段落

In [5]:

%load_ext autoreload
%autoreload 2
%matplotlib inline

recurrent_ppo = RecurrentPPO(
    "MlpLstmPolicy",
    env,
    verbose=1,


)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
!pip install -q optuna
!pip install -q optuna.pruners
!pip install -q optuna.samplers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement optuna.pruners (from versions: none)
ERROR: No matching distribution found for optuna.pruners
ERROR: Could not find a version that satisfies the requirement optuna.samplers (from versions: none)
ERROR: No matching distribution found for optuna.samplers


In [7]:
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(1.2e5)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3


ENV_ID = "Pogema-v0"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpLstmPolicy",
    "verbose": 1,
    "env": ENV_ID
}

def sample_recurrentppo_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for RecurrentPPO hyperparameters."""
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    gae_lambda = 1.0 - trial.suggest_float("gae_lambda", 0.001, 0.2, log=True)
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    ortho_init = trial.suggest_categorical("ortho_init", [False, True])
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    # Display true values.
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("gae_lambda_", gae_lambda)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]} if net_arch == "tiny" else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]


    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "gae_lambda": gae_lambda,
        "learning_rate": learning_rate,
        "ent_coef": ent_coef,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
            "ortho_init": ortho_init,
        },
    }

class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_recurrentppo_params(trial))
    # Create the RL model.
    model = RecurrentPPO(**kwargs)
    # Create env used for evaluation.
    eval_env = Monitor(env)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))

[I 2023-11-10 14:43:06,665] A new study created in memory with name: no-name-2cfdd1ef-342e-4c57-adfd-da5d961ff00e
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you s

Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 42       |
|    ep_rew_mean     | 0.5      |
| time/              |          |
|    fps             | 135      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 1024     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 46.2        |
|    ep_rew_mean          | 0.419       |
| time/                   |             |
|    fps                  | 32          |
|    iterations           | 2           |
|    time_elapsed         | 62          |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.019005027 |
|    clip_fraction        | 0.131       |
|    clip

[I 2023-11-10 16:33:05,571] Trial 0 finished with value: 0.0 and parameters: {'gamma': 0.000947142505866301, 'max_grad_norm': 2.623075679889027, 'gae_lambda': 0.005397763498210882, 'exponent_n_steps': 10, 'lr': 0.0007728363068000152, 'ent_coef': 4.519589464661238e-08, 'ortho_init': True, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 0 with value: 0.0.


Number of finished trials:  1
Best trial:
  Value:  0.0
  Params: 
    gamma: 0.000947142505866301
    max_grad_norm: 2.623075679889027
    gae_lambda: 0.005397763498210882
    exponent_n_steps: 10
    lr: 0.0007728363068000152
    ent_coef: 4.519589464661238e-08
    ortho_init: True
    net_arch: tiny
    activation_fn: tanh
  User attrs:
    gamma_: 0.9990528574941338
    gae_lambda_: 0.9946022365017891
    n_steps: 1024


In [8]:
val_env = gym.make("Pogema-v0",grid_config=grid_config)

val_recurrent_ppo = RecurrentPPO(
     "MlpLstmPolicy",
    val_env,
    verbose=1,
    gamma=0.9946022365017891,
    learning_rate=0.0007728363068000152,



)

mean_reward, std_reward = evaluate_policy(
    val_recurrent_ppo,
    val_recurrent_ppo.get_env(),
    deterministic=True,
    n_eval_episodes=20,
)

print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
mean_reward: 0.00 +/- 0.00


In [9]:
# Train agent and save it
recurrent_ppo.learn(total_timesteps=int(1.2e5))
recurrent_ppo.save("saved/recurrentppo_baseline")

流式输出内容被截断，只能显示最后 5000 行内容。
|    value_loss           | 0.0104     |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 50.8        |
|    ep_rew_mean          | 0.25        |
| time/                   |             |
|    fps                  | 20          |
|    iterations           | 701         |
|    time_elapsed         | 4422        |
|    total_timesteps      | 89728       |
| train/                  |             |
|    approx_kl            | 0.033658184 |
|    clip_fraction        | 0.068       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.489      |
|    explained_variance   | -2          |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0209     |
|    n_updates            | 7000        |
|    policy_gradient_loss | -0.0142     |
|    value_loss           | 0.0116      |
-----------------------------------------
---------

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'saved' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [10]:
def evaluate_success_rate(model, env, num_episodes=100):
    success_count = 0
    step_array = []
    for _ in range(num_episodes):
        obs = env.reset()

        # Check if observation is a tuple and extract the first element if true.
        if isinstance(obs, tuple):
            obs = obs[0]
        max_step = 64
        steps_taken = 0
        done = truncated = False
        while not done and max_step > 0:
            action, _ = model.predict(obs)
            next_obs, reward, done, truncated, info = env.step(action)
            print(action,max_step,success_count,done)
            max_step -= 1
            steps_taken += 1
            # Check if next_obs is a tuple and extract the first element if true.
            if isinstance(next_obs, tuple):
                next_obs = next_obs[0]
            obs = next_obs

            # Check if agent was successful in that episode.
            if done:
                success_count += 1
                step_array.append(steps_taken)
                break

    success_rate = success_count / num_episodes
    return success_rate, step_array

success_rate,step_array = evaluate_success_rate(recurrent_ppo, env)
print(f"Agent Success Rate: {success_rate * 100:.2f}%")
print(f"steps to termination : {step_array}")

# Calculate the average steps to termination
average_steps = sum(step_array) / len(step_array)
print("Average steps to termination:", average_steps)

流式输出内容被截断，只能显示最后 5000 行内容。
3 33 0 False
4 32 0 False
3 31 0 False
4 30 0 False
3 29 0 False
4 28 0 False
3 27 0 False
4 26 0 False
3 25 0 False
4 24 0 False
3 23 0 False
4 22 0 False
3 21 0 False
4 20 0 False
3 19 0 False
4 18 0 False
3 17 0 False
4 16 0 False
3 15 0 False
4 14 0 False
3 13 0 False
4 12 0 False
3 11 0 False
4 10 0 False
3 9 0 False
4 8 0 False
3 7 0 False
4 6 0 False
3 5 0 False
4 4 0 False
2 3 0 False
3 2 0 False
4 1 0 False
3 64 0 False
4 63 0 False
4 62 0 False
3 61 0 False
2 60 0 False
2 59 0 False
2 58 0 False
3 57 0 False
3 56 0 False
3 55 0 False
3 54 0 False
3 53 0 False
2 52 0 False
3 51 0 False
3 50 0 False
3 49 0 False
3 48 0 False
3 47 0 False
3 46 0 False
4 45 0 False
3 44 0 False
3 43 0 False
0 42 0 False
3 41 0 False
3 40 0 False
3 39 0 False
3 38 0 False
2 37 0 False
3 36 0 False
3 35 0 False
3 34 0 False
3 33 0 False
2 32 0 False
3 31 0 False
3 30 0 False
2 29 0 False
3 28 0 False
2 27 0 False
3 26 0 False
2 25 0 False
3 24 0 False
3 23 0 False
3 22 0 